In [ ]:
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import plotly.express as px
import numpy as np
from librerie.fuzioni import *

Estraggo i file

In [ ]:
movies_df = pd.read_csv("C:/Users/bland/PycharmProjects/progetto/data/movies.csv")
genres_df = pd.read_csv("C:/Users/bland/PycharmProjects/progetto/data/genres.csv")
countries_df = pd.read_csv("C:/Users/bland/PycharmProjects/progetto/data/countries.csv")
countries_gdf= gpd.read_file("C:/Users/bland/PycharmProjects/progetto/Country Data/ne_10m_admin_0_countries.shp")

## pulizia movies

In [ ]:
movies_df.shape

In [ ]:
movies_df.head()

In [ ]:
movies_df = movies_df.set_index('id')
movies_df.info()

In [ ]:
movies_df.describe()

In [ ]:
movies_df['name'] = movies_df['name'].astype('string')
movies_df['tagline'] = movies_df['tagline'].astype('string')
movies_df['description'] = movies_df['description'].astype('string')

In [ ]:
movies_df['year'] = pd.to_datetime(movies_df['date'], format='%Y')


In [ ]:
movies_df = movies_df.drop('date', axis = 1)

In [ ]:
movies_df.info()

In [ ]:
movies_df.tail()

In [ ]:
movies_df.duplicated(subset=['name', 'year', 'minute', 'description', 'tagline', 'rating']).sum()

In [ ]:
movies_df = movies_df.drop_duplicates(subset=['name', 'year', 'minute', 'description', 'tagline', 'rating'])

## Pulizia generi

In [ ]:
genres_df['genre'] = genres_df['genre'].astype('string')

In [ ]:
genres_df = genres_df.drop_duplicates(subset=['id'])

## Pulizia paesi

In [ ]:
countries_df.shape

In [ ]:
countries_df.head()

In [ ]:
print("US?", countries_df['country'].str.contains("Trinidad", na=False).sum())

In [ ]:
f = countries_df['country'].str.contains('Antigua', na=False)
g = ~countries_df['country'].str.contains('', na=False)
h = ~countries_df['country'].str.contains('USA', na=False)
l = ~countries_df['country'].str.contains('USSR', na=False)
a = f & g & h & l
print(countries_df.loc[f, 'country'])

In [ ]:
184182 - 174489

## grafici

In [ ]:
f = movies_df[movies_df['year'] == "2031-01-01"]
f.head()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=movies_df, x='year', bins=20, kde=False, color='skyblue')
plt.title('Distribuzione negli anni')
plt.xlabel('anni')
plt.ylabel('Numero di film')
plt.grid(True)
plt.show()

In [ ]:
max_rating = movies_df['rating'].max()
top_movies = movies_df[movies_df['rating'] == max_rating]
print(top_movies)

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=movies_df, x='rating', bins=20, kde=False, color='skyblue')
plt.title('Distribuzione dei Rating')
plt.xlabel('Rating')
plt.ylabel('Numero di film')
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
sns.violinplot(data=movies_df, x='rating', color='lightblue')
plt.title('Distribuzione dei Rating (Violin Plot)')
plt.xlabel('Rating')
plt.grid(True)
plt.show()


In [ ]:
dup_key = movies_df[movies_df.duplicated(subset=['name', 'year'])].iloc[0][['name', 'year']]
coppia = movies_df[(movies_df['name'] == dup_key['name']) & (movies_df['year'] == dup_key['year'])]

print(coppia)

In [ ]:
genres_df.shape

In [ ]:
genres_df.info()


In [ ]:
genres_df.head(50)

In [ ]:
plt.figure(figsize=(12, 8))
ax = sns.countplot(x = 'genre', data = genres_df)
plt.title('count of movies for genres')
plt.xlabel('genres')
plt.ylabel('Count')
plt.xticks(rotation=90)
counts_on_bars(ax)
plt.show()

In [ ]:
merged_df = pd.merge(movies_df, genres_df, left_on='id', right_on='id', how='right')

In [ ]:
genres_df.shape

In [ ]:
merged_df.head()

In [ ]:
plt.figure(figsize=(12,6))
sns.violinplot(data=merged_df, x='genre', y='rating', inner='quartile', hue = 'genre', palette='pastel')
plt.title('Distribuzione dei Rating per Genere')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(12,6))
sns.violinplot(data=merged_df, x='genre', y='year', inner='quartile', hue = 'genre', palette='pastel')
plt.title('Distribuzione dei Rating per Genere')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
merged_df.head()

In [ ]:
countries_df

#world_data_gdf.plot(figsize=(15,10), edgecolor='black', linewidth=1.5, column='Tot Cases/1M pop', legend=True)

In [ ]:
conteggi = countries_df['country'].value_counts().reset_index()
conteggi.columns = ['country', 'count']
conteggi = conteggi.set_index('country')

In [ ]:
fig = px.choropleth(conteggi,
                    locations=conteggi.index,
                    locationmode='country names',
                    color='count',
                    color_continuous_scale='Blues',
                    title='Distribuzione dei Paesi in base al numero di occorrenze')

fig.show()


In [ ]:
conteggi.head()

In [ ]:
countries_gdf.head()

In [ ]:
riga = conteggi.loc['Serbia and Montenegro']
conteggi = pd.concat([conteggi, riga.to_frame().T])
matching = conteggi.index[conteggi.index == 'Serbia and Montenegro']
if len(matching) > 0:
    idx_list = list(conteggi.index)
    first_match_pos = idx_list.index('Serbia and Montenegro')
    idx_list[first_match_pos] = 'Serbia2'
    conteggi.index = idx_list
conteggi.index = conteggi.index.str.replace('Serbia and Montenegro', 'Montenegro2', regex=False)
conteggi.loc['Serbia', 'count'] += conteggi.loc['Serbia2', 'count']
conteggi = conteggi.drop('Serbia2')
conteggi.loc['Montenegro', 'count'] += conteggi.loc['Montenegro2', 'count']
conteggi = conteggi.drop('Montenegro2')

In [ ]:
riga = conteggi.loc['Czechoslovakia']
conteggi = pd.concat([conteggi, riga.to_frame().T])
matching = conteggi.index[conteggi.index == 'Czechoslovakia']
if len(matching) > 0:
    idx_list = list(conteggi.index)
    first_match_pos = idx_list.index('Czechoslovakia')
    idx_list[first_match_pos] = 'Czechia2'
    conteggi.index = idx_list
conteggi.index = conteggi.index.str.replace('Czechoslovakia', 'Slovakia2', regex=False)
conteggi.loc['Czechia', 'count'] += conteggi.loc['Czechia2', 'count']
conteggi = conteggi.drop('Czechia2')
conteggi.loc['Slovakia', 'count'] += conteggi.loc['Slovakia2', 'count']
conteggi = conteggi.drop('Slovakia2')

In [ ]:
conteggi.index = conteggi.index.str.replace('Russian Federation', 'Russia', regex=False)
conteggi.loc['Russia', 'count'] += conteggi.loc['USSR', 'count']
conteggi = conteggi.drop('USSR')

In [ ]:
conteggi.loc['Norway', 'count'] += conteggi.loc['Svalbard and Jan Mayen', 'count']
conteggi = conteggi.drop('Svalbard and Jan Mayen')
conteggi.loc['Norway', 'count'] += conteggi.loc['Bouvet Island', 'count']
conteggi = conteggi.drop('Bouvet Island')

In [ ]:
conteggi.loc['Australia', 'count'] += conteggi.loc['Christmas Island', 'count']
conteggi = conteggi.drop('Christmas Island')
conteggi.loc['Australia', 'count'] += conteggi.loc['Cocos (Keeling) Islands', 'count']
conteggi = conteggi.drop('Cocos (Keeling) Islands')

In [ ]:
conteggi.loc['New Zealand', 'count'] += conteggi.loc['Tokelau', 'count']
conteggi = conteggi.drop('Tokelau')

In [ ]:
conteggi.loc['Netherlands', 'count'] += conteggi.loc['Netherlands Antilles', 'count']
conteggi = conteggi.drop('Netherlands Antilles')

In [ ]:
conteggi.loc['France', 'count'] += conteggi.loc['French Guiana', 'count']
conteggi = conteggi.drop('French Guiana')
conteggi.loc['France', 'count'] += conteggi.loc['Réunion', 'count']
conteggi = conteggi.drop('Réunion')
conteggi.loc['France', 'count'] += conteggi.loc['Guadeloupe', 'count']
conteggi = conteggi.drop('Guadeloupe')
conteggi.loc['France', 'count'] += conteggi.loc['Martinique', 'count']
conteggi = conteggi.drop('Martinique')
conteggi.loc['France', 'count'] += conteggi.loc['Mayotte', 'count']
conteggi = conteggi.drop('Mayotte')

In [ ]:
conteggi.loc['Germany', 'count'] += conteggi.loc['East Germany', 'count']
conteggi = conteggi.drop('East Germany')

In [ ]:
a = ['Slovenia', 'Croatia', 'North Macedonia', 'Bosnia and Herzegovina', 'Serbia', 'Montenegro', 'Kosovo']
a1 = ['Slovenia1', 'Croatia1', 'North Macedonia1', 'Bosnia and Herzegovina1', 'Serbia1', 'Montenegro1', 'Kosovo1']

riga = conteggi.loc['Yugoslavia']
for i in a:
    conteggi = pd.concat([conteggi, riga.to_frame().T])
for i in a1:
    matching = conteggi.index[conteggi.index == 'Yugoslavia']
    if len(matching) > 0:
        idx_list = list(conteggi.index)
        first_match_pos = idx_list.index('Yugoslavia')
        idx_list[first_match_pos] = i
        conteggi.index = idx_list
conteggi = conteggi.drop('Yugoslavia')
for i, j in zip(a, a1):
    conteggi.loc[i, 'count'] += conteggi.loc[j, 'count']
    conteggi = conteggi.drop(j)

In [ ]:
conteggi.index = conteggi.index.str.replace('USA', 'United States of America', regex=False)
conteggi.index = conteggi.index.str.replace('UK', 'United Kingdom', regex=False)
conteggi.index = conteggi.index.str.replace('Bolivarian Republic of Venezuela', 'Venezuela', regex=False)
conteggi.index = conteggi.index.str.replace('Syrian Arab Republic', 'Syria', regex=False)
conteggi.index = conteggi.index.str.replace('Eswatini', 'eSwatini', regex=False)
conteggi.index = conteggi.index.str.replace('Democratic Republic of Congo', 'Democratic Republic of the Congo', regex=False)
conteggi.index = conteggi.index.str.replace(r'^Congo', 'Republic of the Congo', regex=True)
conteggi.index = conteggi.index.str.replace('Republic of Moldova', 'Moldova', regex=False)
conteggi.index = conteggi.index.str.replace('Serbia', 'Republic of Serbia', regex=False)
conteggi.index = conteggi.index.str.replace('Bahamas', 'The Bahamas', regex=False)
conteggi.index = conteggi.index.str.replace('Timor-Leste', 'East Timor', regex=False)
conteggi.index = conteggi.index.str.replace('State of Palestine', 'Palestine', regex=False)
conteggi.index = conteggi.index.str.replace('Cape Verde', 'Cabo Verde', regex=False)
conteggi.index = conteggi.index.str.replace('US Virgin Islands', 'United States Virgin Islands', regex=False)
conteggi.index = conteggi.index.str.replace("Lao People's Democratic Republic", 'Laos', regex=False)
conteggi.index = conteggi.index.str.replace("Sao Tome and Principe", 'São Tomé and Principe', regex=False)
conteggi.index = conteggi.index.str.replace('Vatican City', 'Vatican', regex=False)
conteggi.index = conteggi.index.str.replace('Hong Kong', 'Hong Kong S.A.R.', regex=False)
conteggi.index = conteggi.index.str.replace('Brunei Darussalam', 'Brunei', regex=False)
conteggi.index = conteggi.index.str.replace('Macao', 'Macao S.A.R', regex=False)
conteggi.index = conteggi.index.str.replace('South Georgia and the South Sandwich Islands', 'South Georgia and the Islands', regex=False)
conteggi.index = conteggi.index.str.replace('Pitcairn', 'Pitcairn Islands', regex=False)
conteggi.index = conteggi.index.str.replace('Saint Helena, Ascension and Tristan da Cunha', 'Saint Helena', regex=False)
conteggi.index = conteggi.index.str.replace('French Southern Territories', 'French Southern and Antarctic Lands', regex=False)

In [ ]:
conteggi.loc['Baykonur Cosmodrome', 'count'] = conteggi.loc['Kazakhstan', 'count']
conteggi.loc['Brazilian Island', 'count'] = conteggi.loc['Brazil', 'count']
conteggi.loc['Isle of Man', 'count'] = conteggi.loc['United Kingdom', 'count']
conteggi.loc['Somaliland', 'count'] = conteggi.loc['Somalia', 'count']
conteggi.loc['Cyprus No Mans Area', 'count'] = conteggi.loc['Cyprus', 'count']
conteggi.loc['Northern Cyprus', 'count'] = conteggi.loc['Turkey', 'count']
conteggi.loc['US Naval Base Guantanamo Bay', 'count'] = conteggi.loc['Cuba', 'count']
conteggi.loc['Sint Maarten', 'count'] = conteggi.loc['Netherlands', 'count']
conteggi.loc['Saint Martin', 'count'] = conteggi.loc['France', 'count']
conteggi.loc['Saint Barthelemy', 'count'] = conteggi.loc['France', 'count']
conteggi.loc['Clipperton Island', 'count'] = conteggi.loc['France', 'count']
conteggi.loc['Dhekelia Sovereign Base Area', 'count'] = conteggi.loc['United Kingdom', 'count']
conteggi.loc['Akrotiri Sovereign Base Area', 'count'] = conteggi.loc['United Kingdom', 'count']
conteggi.loc['Indian Ocean Territories', 'count'] = conteggi.loc['Australia', 'count']
conteggi.loc['Jersey', 'count'] = conteggi.loc['United Kingdom', 'count']
conteggi.loc['Guernsey', 'count'] = conteggi.loc['United Kingdom', 'count']
conteggi.loc['Coral Sea Islands', 'count'] = conteggi.loc['Australia', 'count']
conteggi.loc['Ashmore and Cartier Islands', 'count'] = conteggi.loc['Australia', 'count']
conteggi.loc['Aland', 'count'] = conteggi.loc['Finland', 'count']
conteggi.loc['Curaçao', 'count'] = conteggi.loc['Netherlands', 'count']
conteggi.loc['Bajo Nuevo Bank (Petrel Is.)', 'count'] = conteggi.loc['Colombia', 'count']
conteggi.loc['Serranilla Bank', 'count'] = conteggi.loc['Colombia', 'count']
conteggi.loc['Siachen Glacier', 'count'] = conteggi.loc['India', 'count']
conteggi.loc['Southern Patagonian Ice Field', 'count'] = conteggi.loc['Chile', 'count']
conteggi.loc['Scarborough Reef', 'count'] = conteggi.loc['Philippines', 'count']

In [ ]:
world_data_gdf = countries_gdf.merge(conteggi, left_on='ADMIN', right_on=conteggi.index, how='left')
world_data_gdf.head()

In [ ]:
Rome= [41.9028, 12.4964]
world_data_gdf['count_log'] = np.log1p(world_data_gdf['count'])
m = folium.Map(location=Rome, zoom_start=2)
choropleth = folium.Choropleth(
    geo_data=world_data_gdf,
    data=world_data_gdf,
    columns=['ADMIN', 'count_log'],
    key_on='feature.properties.ADMIN',
    fill_color="Reds",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='numero film'
).add_to(m)
m

In [ ]:
filtro = conteggi.index.str.contains('maa', case=False, na=False) #| conteggi.index.str.contains('croatia', case=False, na=False) | conteggi.index.str.contains('koso', case=False, na=False) | conteggi.index.str.contains('bosnia', case=False, na=False) | conteggi.index.str.contains('sloveni', case=False, na=False) | conteggi.index.str.contains('maced', case=False, na=False) | conteggi.index.str.contains('serb', case=False, na=False)| conteggi.index.str.contains('sla', case=False, na=False)
conteggi.loc[filtro]


In [ ]:
filtro = world_data_gdf['SOVEREIGNT'].str.contains('Southern Patagonian Ice Field', case=False, na=False)
world_data_gdf.loc[filtro]

In [ ]:
#Croatia,1276
#Republic of Serbia,1369
#Slovenia,1006
#Bosnia and Herzegovina,533
#North Macedonia,525
#Kosovo,138
#Montenegro,222
#2411

In [ ]:
filtro = ~conteggi.index.isin(world_data_gdf['ADMIN'])
risultato = conteggi[filtro]
print(risultato)


In [ ]:
filtro = ~world_data_gdf['ADMIN'].isin(conteggi.index)
risultato = world_data_gdf['ADMIN'][filtro]
print(risultato)

In [ ]:
filtro = world_data_gdf['count'].isna()
risultato = world_data_gdf['NAME'][filtro]
print(risultato)

In [ ]:
conteggi.index.duplicated().sum()

In [ ]:
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Brazilian Island', 'NOTE_BRK'].values,'Brazil')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Somaliland', 'NOTE_BRK'].values,'Somalia')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Cyprus No Mans Area', 'NOTE_BRK'].values,'Cyprus')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Northern Cyprus', 'NOTE_BRK'].values, 'Turkey')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Bajo Nuevo Bank (Petrel Is.)', 'NOTE_BRK'].values, 'Colombia')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Serranilla Bank', 'NOTE_BRK'].values, 'Colombia')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Siachen Glacier', 'NOTE_BRK'].values, 'India')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Southern Patagonian Ice Field', 'NOTE_BRK'].values, 'Chile')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Scarborough Reef', 'NOTE_BRK'].values, 'Philippines')

print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Spratly Is.', 'NOTE_BRK'].values, ' ')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Bir Tawil', 'NOTE_BRK'].values, ' ')

In [ ]:
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Baykonur Cosmodrome', 'NOTE_BRK'].values, 'Kazakhstan')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Isle of Man', 'NOTE_BRK'].values,'United Kingdom')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'US Naval Base Guantanamo Bay', 'NOTE_BRK'].values, 'Cuba')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Sint Maarten', 'NOTE_BRK'].values, 'Netherlands')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Saint Martin', 'NOTE_BRK'].values, 'France')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Clipperton Island', 'NOTE_BRK'].values, 'France')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Saint Barthelemy', 'NOTE_BRK'].values, 'France')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Dhekelia Sovereign Base Area', 'NOTE_BRK'].values, 'United Kingdom')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Akrotiri Sovereign Base Area', 'NOTE_BRK'].values, 'United Kingdom')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Indian Ocean Territories', 'NOTE_BRK'].values, 'Australia')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Jersey', 'NOTE_BRK'].values, 'United Kingdom')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Guernsey', 'NOTE_BRK'].values, 'United Kingdom')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Coral Sea Islands', 'NOTE_BRK'].values, 'Australia')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Ashmore and Cartier Islands', 'NOTE_BRK'].values, 'Australia')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Aland', 'NOTE_BRK'].values, 'Finland')
print(world_data_gdf.loc[world_data_gdf['ADMIN'] == 'Curaçao', 'NOTE_BRK'].values, 'Netherlands')


In [ ]:
conteggi.tail(40)